In [11]:
from jyro.simulator import (Robot, Pioneer, Pioneer16Sonars, PioneerFrontLightSensors,
                            Camera, Simulator, VSimulator)
import numpy as np
from math import pi
from random import random

In [12]:

def make_world(physics):
    physics.addBox(0, 0, 4, 4, fill="backgroundgreen", wallcolor="gray")
    physics.addBox(1.75, 2.9, 2.25, 3.0, fill="blue", wallcolor="blue")
    physics.addLight(2, 3.5, 1.0)

In [13]:
help(Pioneer)

Help on class Pioneer in module jyro.simulator.robot:

class Pioneer(Robot)
 |  Pioneer(name, x, y, a, color='red')
 |  
 |  Method resolution order:
 |      Pioneer
 |      Robot
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, x, y, a, color='red')
 |      a - angle in radians
 |  
 |  drawRobot(self, canvas)
 |      Draws the body of the robot. Not very efficient.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Robot:
 |  
 |  __getitem__(self, item)
 |  
 |  addBoundingSeg(self, boundingSeg)
 |  
 |  addDevice(self, dev)
 |  
 |  additionalSegments(self, propose, x, y, cos_a90, sin_a90, **dict)
 |      propose is where it would go, if moved with device velocity, etc.
 |  
 |  brain(self, robot)
 |  
 |  draw(self, canvas)
 |  
 |  drawDevices(self, canvas)
 |  
 |  drawRay(self, dtype, x1, y1, x2, y2, color)
 |  
 |  getIndex(self, dtype, i)
 |  
 |  getPose(self)
 |      Returns global c

In [4]:
def make_robot():
    robot = Pioneer("Pioneer", 2, 1, 0) #paremeters are x, y, heading (in radians)
    robot.addDevice(Camera())
    robot.addDevice(Pioneer16Sonars())
    light_sensors = PioneerFrontLightSensors(3) #parameter defines max range
    #light_sensors.lightMode = 'ambient'
    robot.addDevice(light_sensors)
    return robot

In [14]:
def get_senses(robot):
    light = robot["light"].getData()
    #print("light", light)
    sonar = robot["sonar"].getData()
    #print("sonar", sonar)
    return [light, sonar]

In [15]:

def random_start(robot):
    robot.setPose(0.5 + random()*2.5, 0.5 + random()*2, random()*2*pi)

In [16]:

def determine_move(senses):
    """Returns tuple of (translation, rotation) movement"""
    lights = senses[0]
    left_light = lights[0]
    right_light = lights[1]
    light_diff = abs(left_light-right_light)
    sonars = senses[1]
    # if found light, then stop
    if sum(lights) > 1.8:
        return (0, 0)
    # if getting close to an obstacle in front, turn to avoid it
    elif min(sonars[2:6]) < 0.5:
        # if closer on left, turn right
        if min(sonars[1:4]) < min(sonars[4:7]):
            return (0, -0.3)
        # otherwise, turn left
        else:
            return (0, 0.3)
    # if diff in light readings is high enough or total of light readings is
    # low ennough, then turn towards the light
    elif light_diff > 0.1 or sum(lights) < 0.1:
        # if brighter on left side, turn slightly left
        if  left_light > right_light:
            return (0.1, 0.3)
        else:
            return (0.1, -0.3)
    # default is to go straight
    else:
        return (0.3, 0)

def find_light_brain(robot):
    senses = get_senses(robot)
    translate, rotate = determine_move(senses)
    robot.move(translate, rotate)

In [17]:
robot = make_robot()
vsim = VSimulator(robot, make_world) #create a visual simulator to watch robot's behavior
random_start(robot)
vsim.update_gui()
robot.brain = find_light_brain

In [18]:
def generate_data(robot, make_world, trials, filename):
    sim = Simulator(robot, make_world)
    fp = open(filename, "w")
    for i in range(trials):
        print("Trial %d" % i)
        random_start(robot)
        while True:
            senses = get_senses(robot)
            print(senses)
            translate, rotate = determine_move(senses)
            if translate == 0 and rotate == 0:
                break # found light, so end trial
            robot.move(translate, rotate)
            sim.step()
            lights = senses[0]
            sonars = [min(v/3.0, 1.0) for v in senses[1]] #normalize sonar values
            for value in lights:
                fp.write("%.3f " % value)
            for value in sonars[1:7]:
                fp.write("%.3f " % value)
            fp.write("%.1f %.1f\n" % (translate, rotate))
    fp.close()

In [10]:
generate_data(robot, make_world, 5, "testing_data.txt")

Trial 0
[[0.28802020846300447, 0.23222543721892297], [1.807914115605531, 0.7145828145779953, 2.3732444613621455, 2.403686265619226, 2.1632742711188166, 2.4987760829979324, 2.3438896116498853, 1.9071772464798886, 1.8709498694524394, 1.91611360495301, 2.2474633366158234, 1.549556650293022, 1.3882178965537544, 1.609253758204979, 2.2679206067734228, 1.8441415104566052]]
[[0.2896058901812595, 0.2334789567477666], [1.8024799952196642, 0.7090539526331687, 2.366291464183322, 2.371043685148776, 2.133653671527357, 2.46478089027602, 2.3504386350960815, 1.9126112506433606, 1.8763839720146833, 1.9216424668978385, 2.293644662995317, 1.5821992307634711, 1.4178385043963928, 1.6432489509268917, 2.2613715255884204, 1.8387073900707367]]
[[0.2910839119044629, 0.234644170953285], [1.7970459910561916, 0.7035250652569105, 2.3593384030834685, 2.338401104678327, 2.1040330636847178, 2.430785697554108, 2.356987716281084, 1.9180453710292282, 1.8818180924005514, 1.9271712575619324, 2.3398261431812086, 1.6148419016